In [1]:
import numpy as np
import random

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def matmul(A, B):
    result = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return np.array(result)

Neural Network

In [2]:
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        self.W1 = np.random.rand(input_size, hidden1_size)
        self.b1 = np.zeros((1, hidden1_size))

        self.W2 = np.random.rand(hidden1_size, hidden2_size)
        self.b2 = np.zeros((1, hidden2_size))

        self.W3 = np.random.rand(hidden2_size, output_size)
        self.b3 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = matmul(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)

        self.z2 = matmul(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)

        self.z3 = matmul(self.a2, self.W3) + self.b3
        self.a3 = sigmoid(self.z3)

        return self.a3

    def backward(self, X, y, output, learning_rate):
        output_error = output - y
        output_delta = output_error * sigmoid_derivative(output)

        hidden2_error = matmul(output_delta, self.W3.T)
        hidden2_delta = hidden2_error * sigmoid_derivative(self.a2)

        hidden1_error = matmul(hidden2_delta, self.W2.T)
        hidden1_delta = hidden1_error * sigmoid_derivative(self.a1)

        self.W3 -= matmul(self.a2.T, output_delta) * learning_rate
        self.b3 -= np.sum(output_delta, axis=0, keepdims=True) * learning_rate

        self.W2 -= matmul(self.a1.T, hidden2_delta) * learning_rate
        self.b2 -= np.sum(hidden2_delta, axis=0, keepdims=True) * learning_rate

        self.W1 -= matmul(X.T, hidden1_delta) * learning_rate
        self.b1 -= np.sum(hidden1_delta, axis=0, keepdims=True) * learning_rate

Data

In [3]:
def flip_pixels(image, flip_rate=0.2):
    flipped = image.copy()
    num_flips = int(len(image) * flip_rate)
    indices = random.sample(range(len(image)), num_flips)
    for i in indices:
        flipped[i] = 1 - flipped[i]
    return flipped

In [4]:
X = np.array([
    [0,1,1,0,1,0,1,0,0],
    [1,1,0,1,0,1,0,1,1],
    [1,0,1,0,1,0,1,0,1],
    [1,1,1,1,0,1,1,1,1],
    [0,0,1,0,1,0,1,0,0]
])

y = np.array([
    [1,0,0,0,0],
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,0,1]
])

X_noisy = np.array([flip_pixels(img, 0.2) for img in X])
X_total = np.vstack((X, X_noisy))
y_total = np.vstack((y, y))

In [5]:
nn = NeuralNetwork(input_size=9, hidden1_size=6, hidden2_size=4, output_size=5)

for epoch in range(1000):
    for i in range(len(X_total)):
        out = nn.forward(X_total[i].reshape(1, -1))
        nn.backward(X_total[i].reshape(1, -1), y_total[i].reshape(1, -1), out, learning_rate=0.1)

test_input = X[0].reshape(1, -1)
prediction = nn.forward(test_input)
print("Prediction:", prediction)
print("Most likely class:", np.argmax(prediction))

Prediction: [[0.41357794 0.14579094 0.2188718  0.1212128  0.33974379]]
Most likely class: 0
